In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
products_df = pd.read_csv('/kaggle/input/amazon-products-dataset-2023-1-4m-products/amazon_products.csv')
categories_df = pd.read_csv('/kaggle/input/amazon-products-dataset-2023-1-4m-products/amazon_categories.csv')


In [3]:
# Kết hợp dữ liệu từ hai DataFrame bằng cột category_id và id
merged_df = pd.merge(products_df, categories_df, left_on='category_id', right_on='id')

# Xóa cột 'category_id' cuối cùng
merged_df.drop(columns=['category_id'], inplace=True)

In [4]:
print(merged_df.columns)


Index(['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price',
       'listPrice', 'isBestSeller', 'boughtInLastMonth', 'id',
       'category_name'],
      dtype='object')


In [5]:
print(merged_df.head())


         asin                                              title  \
0  B014TMV5YE  Sion Softside Expandable Roller Luggage, Black...   
1  B07GDLCQXV  Luggage Sets Expandable PC+ABS Durable Suitcas...   
2  B07XSCCZYG  Platinum Elite Softside Expandable Checked Lug...   
3  B08MVFKGJM  Freeform Hardside Expandable with Double Spinn...   
4  B01DJLKZBA  Winfield 2 Hardside Expandable Luggage with Sp...   

                                              imgUrl  \
0  https://m.media-amazon.com/images/I/815dLQKYIY...   
1  https://m.media-amazon.com/images/I/81bQlm7vf6...   
2  https://m.media-amazon.com/images/I/71EA35zvJB...   
3  https://m.media-amazon.com/images/I/91k6NYLQyI...   
4  https://m.media-amazon.com/images/I/61NJoaZcP9...   

                             productURL  stars  reviews   price  listPrice  \
0  https://www.amazon.com/dp/B014TMV5YE    4.5        0  139.99       0.00   
1  https://www.amazon.com/dp/B07GDLCQXV    4.5        0  169.99     209.99   
2  https://www.amazo

In [6]:
# Loại bỏ các hàng có giá trị NaN trong cột 'title'
merged_df.dropna(subset=['title'], inplace=True)

In [7]:
# Chọn các cột cần thiết cho mô hình
product_titles = merged_df['title']

In [8]:
# Bước 1: Chuẩn bị dữ liệu với TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(product_titles)


In [9]:
print("TF-IDF matrix shape:", tfidf_matrix.shape)


TF-IDF matrix shape: (1426336, 428801)


In [10]:
# Bước 2: Huấn luyện mô hình NearestNeighbors
nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
nn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [11]:
# Bước 3: Tạo hàm gợi ý sản phẩm
def recommend_products(product_name, num_recommendations=5):
    # Chuyển đổi tên sản phẩm sang vector TF-IDF
    query_vector = vectorizer.transform([product_name])
    
    # Tìm kiếm các sản phẩm tương tự
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=num_recommendations)
    
    # Lấy thông tin các sản phẩm được gợi ý
    recommendations = merged_df.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]
    
    return recommendations[['title', 'stars', 'price', 'category_name', 'distance']]


In [12]:
# Thử nghiệm hàm gợi ý sản phẩm
product_name = "Xpedition 30 Inch Multi-Pocket Upright Rolling Duffel Bag "
recommended_products = recommend_products(product_name, num_recommendations=5)
print(recommended_products)

                                                    title  stars  price  \
373016  Xpedition 30 Inch Multi-Pocket Upright Rolling...    4.3  31.84   
14      Xpedition 30 Inch Multi-Pocket Upright Rolling...    4.3  42.00   
373208        8 Pocket Rolling Duffel Bag, Black, 22 inch    4.5  64.09   
373299              Rolling Duffel Bag, Charcoal, 30-Inch    4.0  35.49   
372997                                 Rolling Duffel Bag    4.2  20.49   

             category_name  distance  
373016  Travel Duffel Bags  0.000000  
14               Suitcases  0.000000  
373208  Travel Duffel Bags  0.441438  
373299  Travel Duffel Bags  0.450364  
372997  Travel Duffel Bags  0.452113  


In [13]:
# Bước 3: Tạo hàm gợi ý sản phẩm theo ID và kèm theo hình ảnh
def recommend_products_by_id(product_id, num_recommendations=5):
    # Lấy tên sản phẩm từ ID
    product_name = merged_df.loc[merged_df['asin'] == product_id, 'title'].values[0]
    
    # Chuyển đổi tên sản phẩm sang vector TF-IDF
    query_vector = vectorizer.transform([product_name])
    
    # Tìm kiếm các sản phẩm tương tự
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=num_recommendations)
    
    # Lấy thông tin các sản phẩm được gợi ý
    recommendations = merged_df.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]
    
    return recommendations[['title', 'stars', 'price', 'category_name', 'imgUrl', 'distance']]



In [14]:
from IPython.display import display, HTML

def recommend_products_by_id(product_id, num_recommendations=5):
    # Lấy tên sản phẩm từ ID
    product_name = merged_df.loc[merged_df['asin'] == product_id, 'title'].values[0]
    
    # Chuyển đổi tên sản phẩm sang vector TF-IDF
    query_vector = vectorizer.transform([product_name])
    
    # Tìm kiếm các sản phẩm tương tự
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=num_recommendations)
    
    # Lấy thông tin các sản phẩm được gợi ý
    recommendations = merged_df.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]
    
    # Hiển thị thông tin sản phẩm và hình ảnh
    for index, row in recommendations.iterrows():
        display(HTML(f"<div style='border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin-bottom: 20px;'>"))
        display(HTML(f"<h3>{row['title']}</h3>"))
        display(HTML(f"<p><b>Stars:</b> {row['stars']}</p>"))
        display(HTML(f"<p><b>Price:</b> ${row['price']}</p>"))
        display(HTML(f"<p><b>Category:</b> {row['category_name']}</p>"))
        display(HTML(f'<img src="{row["imgUrl"]}" alt="{row["title"]}" style="width:200px;height:200px; margin-top: 10px;">'))
        display(HTML(f"<p style='margin-top: 10px;'><b>Distance:</b> {row['distance']}</p>"))
        display(HTML("</div>"))


In [15]:

# Thử nghiệm hàm gợi ý sản phẩm theo ID
product_id = "B01DJLKZBA"  # Thay thế bằng ID sản phẩm cụ thể
recommend_products_by_id(product_id, num_recommendations=5)


In [16]:
from IPython.display import display, HTML

def recommend_products_by_name(product_name, num_recommendations=5):
    # Chuyển đổi tên sản phẩm sang vector TF-IDF
    query_vector = vectorizer.transform([product_name])
    
    # Tìm kiếm các sản phẩm tương tự
    distances, indices = nn_model.kneighbors(query_vector, n_neighbors=num_recommendations)
    
    # Lấy thông tin các sản phẩm được gợi ý
    recommendations = merged_df.iloc[indices[0]].copy()
    recommendations['distance'] = distances[0]
    
    # Hiển thị thông tin sản phẩm và hình ảnh
    for index, row in recommendations.iterrows():
        display(HTML(f"<div style='border: 1px solid #ccc; border-radius: 5px; padding: 10px; margin-bottom: 20px;'>"))
        display(HTML(f"<h3>{row['title']}</h3>"))
        display(HTML(f"<p><b>Stars:</b> {row['stars']}</p>"))
        display(HTML(f"<p><b>Price:</b> ${row['price']}</p>"))
        display(HTML(f"<p><b>Category:</b> {row['category_name']}</p>"))
        display(HTML(f'<img src="{row["imgUrl"]}" alt="{row["title"]}" style="width:200px;height:200px; margin-top: 10px;">'))
        display(HTML(f"<p style='margin-top: 10px;'><b>Distance:</b> {row['distance']}</p>"))
        display(HTML("</div>"))



In [17]:
# Sử dụng hàm với tên sản phẩm cụ thể
product_name = "Xpedition 30 Inch Multi-Pocket Upright Rolling Duffel Bag"  # Thay thế bằng tên sản phẩm cụ thể
recommend_products_by_name(product_name, num_recommendations=5)
